This notebook is to determine the required laser power to illuminate the flat field screen.
This uses a single laser that is positioned on the dome. The light then goes into 15 meters of fiber to the center of the screen. Light is sent off a central reflector and down to the telescope.

Currently in Python3 

In [ ]:
import numpy as np
import math

import matplotlib.pyplot as plt
import scipy.interpolate

In [ ]:
# declare constants
h = 6.626e-34 # m2 kg / s
c = 3e8 # m/s

In [ ]:
# Load EKSPLA NT 242-SHG laser output profile
# no actual profile, just guessing from the curve in the PDF file 
#nt242_wave =      np.array([300,320,350,399,400,420,480,500,540,610,700,800,950,1000,1200,1300.0]) # nm
#nt242_energy_J =  np.array([ 20, 45, 30, 10, 45,470,460,460,300,180, 60,130,155, 160, 150,140])*1e-6 # in joules

nt242_data = np.genfromtxt('Laser_power_data/PGD151_NT242.txt',
                          delimiter='',comments='#') # wavelength [nm], Transmission

nt242_wave = nt242_data[21::,0]
nt242_energy_J = nt242_data[21::,1]*1e-6

#print nt242_wave
#print nt242_energy_J

nt242_pulse_rate_Hz = 1000.0 # 1000 pulses per second

nt242_flux_W = nt242_energy_J * nt242_pulse_rate_Hz # watts

In [ ]:
# Load EKSPLA NT 230-SHG laser output profile
# no actual profile, just guessing from the curve in the PDF file 

#nt230_sfg_data = np.genfromtxt('/Users/patrickingraham/LSST/laser_throughput_calc/Laser_power_data/nt230_sfg.csv',
#                          delimiter=',',comments='#') # wavelength [nm], Transmission
#nt230_signal_data = np.genfromtxt('/Users/patrickingraham/LSST/laser_throughput_calc/Laser_power_data/nt230_signal.csv',
#                          delimiter=',',comments='#') # wavelength [nm], Transmission
#nt230_idler_data = np.genfromtxt('/Users/patrickingraham/LSST/laser_throughput_calc/Laser_power_data/nt230_idler.csv',
#                          delimiter=',',comments='#') # wavelength [nm], Transmission

nt230_data_405nm = np.genfromtxt('Laser_power_data/PGD149_NT230_210nm_405nm.txt',
                          delimiter='',comments='#') # wavelength [nm], Transmission
nt230_data_709nm = np.genfromtxt('Laser_power_data/PGD149_NT230_405nm_709nm.txt',
                          delimiter='',comments='#') # wavelength [nm], Transmission
nt230_data_2600nm = np.genfromtxt('Laser_power_data/PGD149_NT230_710nm_2600nm.txt',
                          delimiter='',comments='#') # wavelength [nm], Transmission

nt230_wave0 =      np.concatenate((nt230_data_405nm[:,0],nt230_data_709nm[:,0],nt230_data_2600nm[:,0])) # nm
nt230_energy_J0 =  np.concatenate((nt230_data_405nm[:,1],nt230_data_709nm[:,1],nt230_data_2600nm[:,1]))*1e-3 # in joules
    
nt230_wave=nt230_wave0[20:95]    
nt230_energy_J = nt230_energy_J0[20:95]
    
nt230_pulse_rate_Hz = 100.0 # 100 pulses per second

nt230_flux_W = nt230_energy_J * nt230_pulse_rate_Hz # watts

### Load laser output profile

In [ ]:
laser = 'nt242_pure'
if laser == 'nt242':
    laser_flux_W = nt242_flux_W
    laser_wave = nt242_wave
    laser_energy_J = nt242_energy_J
if laser == 'nt242_pure':
    laser_flux_W = nt242_flux_W * 0.7 # 30% absorbed from cleaning
    laser_wave = nt242_wave
    laser_energy_J = nt242_energy_J * 0.7
if laser == 'nt230':
    laser_flux_W = nt230_flux_W
    laser_wave = nt230_wave
    laser_energy_J = nt230_energy_J

if 1 == 1:
    if laser == 'nt242' or laser == 'nt242_pure':
        conv_fact=1e6
        plt.ylim([0,500])
        plt.xlim([300,1100])
        plt.ylabel('Pulse Energy, (uJ)')
    if laser == 'nt230':
        conv_fact=1e3
        plt.ylim([0,11])
        plt.ylabel('Pulse Energy, (mJ)')
    plt.plot(laser_wave,laser_energy_J*conv_fact,'-g',label="Power vs wavelengh")
    plt.title('Spectral resolution function')
    
    plt.xlabel('wavelength [nm]')
    plt.legend()
    plt.show()
    plt.close()
    
# assume 100 percent effiency and convert to photons
ind=30 # indice of interest
number_of_photons = laser_flux_W / (h*c/(laser_wave*1e-9))  # photons per second
print("%.4g photons at %d nm" % (number_of_photons[ind], laser_wave[ind]) )

### Load beam transfer bench profile

### Bench is Laser -> fiber

In [ ]:
# tunable_ND_filter_min_throughput = 0.96**2
laser_flux_incident_on_fiber = laser_flux_W # watts

In [ ]:
fiber_coupling_efficiency = 0.5 # from vendor measurement
print("Assuming %0.4g loss on fiber illumination" % fiber_coupling_efficiency )
Flux_into_fiber = laser_flux_incident_on_fiber*(1-fiber_coupling_efficiency) # watts
print(f'Flux into fiber is {Flux_into_fiber[ind]:0.4f} Watts')

In [ ]:
# Fiber efficiency - assuming 15 meters length
# plots are in weird units (dB/km) - want to put that into transmission per meter.
# data from http://www.ceramoptec.de/products/fibers/optran-uv-wf.html

fiber_distance = 15 # meters
fiber_wave= np.array([300,350,400,500,600,700,800,900,950,1000,1100.0,1200.0]) # in nm
fiber_att_db_km0 = np.array([140, 70, 35, 15,8.5,  6,  5,  5,6.5, 3.5,   3,2]) # in db/km
# interpolate to have the same wavelengths as nt_242
fiber_att_db_km = scipy.interpolate.griddata(fiber_wave, fiber_att_db_km0, laser_wave, method='linear')
# convert to transmission over the distance
fiber_att = (10**(-fiber_att_db_km/1000*fiber_distance/10.0)) # transmission per m
fiber_att_80 = (10**(-fiber_att_db_km/1000*80.0/10.0)) # transmission per m
#print(laser_wave)
#print(fiber_att)
if 0 == 1:
    plt.plot(laser_wave,fiber_att,'-g')
    plt.plot(laser_wave,fiber_att_80,'-r')
    #plt.plot(fiber_wave, fiber_att_db_km0,'-b')
    plt.title( 'Fiber Transmission over %d meters' % fiber_distance)
    plt.ylabel('Fiber Transmission over %d meters' % fiber_distance)
    plt.xlabel('wavelength [nm]')
    plt.xlim([300,1200])
    plt.legend()
    plt.ylim([0,1])
    plt.show()
    plt.close()

Flux_out_of_fiber = Flux_into_fiber * fiber_att # watts

print("fiber attenuation is %0.3f at %d nm" % (fiber_att[ind],laser_wave[ind]) )

### Integration Sphere Efficiency

In [ ]:
# Sphere efficiency - from labsphere theory and app document (found in references folder of this repo)
# https://www.labsphere.com/wp-content/uploads/2021/09/GPS.pdf

# Assume 4P-GPS-060-SF AS-02266-060 
# Spectraflect, 6 inch diameter sphere with two ports, each with a 2.5 entrance/exit pupil
sphere_radius = (6.0/2)*0.0254 # in meters
input_port_radius = (2.5/2) * 0.0254 # in meters
output_port_radius = (2.5/2) * 0.0254 # in meters
A_sphere = 4.0 * np.pi * (sphere_radius**2)# area of the sphere
A_input = np.pi * (input_port_radius**2)# input port area
A_exit = np.pi * (output_port_radius**2)# exit port area
f = (A_input + A_exit)/A_sphere # port fraction = 0.087

# assume labsphere AS-02284-033 , 3.3 inch diameter sphere with spectralon. 2 output ports - each 1.5 inch diameter
rho= 0.985  # spectralon reflectance - actually has a very slight wavelength dependence - will put that in later

# sphere_radius = (3.3/2)*0.0254 # in meters
# input_port_radius = (1.5/2 ) * 0.0254 # in meters
# output_port_radius = (1.5/2 ) * 0.0254 # in meters
# A_sphere = 4.0 * np.pi * (sphere_radius**2)# area of the sphere
# A_input = np.pi * (input_port_radius**2)# input port area
# A_exit = np.pi * (output_port_radius**2)# exit port area
# f = (A_input + A_exit)/A_sphere # port fraction = 0.045
# # efficiency = 1.33e-5


# #assume labsphere AS-02281-010, 1 inch radius sphere with spectralon. 2 output ports - each 1 inch diameter
# rho= 0.985  # spectralon reflectance - actually has a very slight wavelength dependence - will put that in later
# sphere_radius = (1/2.)*0.0254 # in meters
# input_port_radius = (0.25/2 ) * 0.0254 # in meters
# output_port_radius = (0.25/2 ) * 0.0254 # in meters
# A_sphere = 4.0 * np.pi * (sphere_radius**2)# area of the sphere
# A_input = np.pi * (input_port_radius**2)# input port area
# A_exit = np.pi * (output_port_radius**2)# exit port area
# f = (A_input + A_exit)/A_sphere # port fraction
laser_flux_into_int_sphere = Flux_out_of_fiber
Ls = ( laser_flux_into_int_sphere / (np.pi * A_sphere) ) * ( rho / (1- rho*(1-f)) ) # eq 12 - Watts/m2
print(f'The amount of light exiting the sphere is {Ls[ind]:0.3f} [Watts/m2]')

In [ ]:
# Mask is ~3 inches from the sphere
# The amount of light incident on the mask is:
d=3*25.4e-3 # [m] distance to mask from exit pupil to focal plane mask (ESTIMATE)
theta = np.arctan(output_port_radius/d)
incident_flux = Ls * np.pi * np.sin(theta)**2 # Watts/m2  
print(f'Incident flux on mask is {incident_flux[ind]:0.3f} [Watts/m2]')

In [ ]:
# calculate how much flux is absorbed by the sphere
mask_area = np.pi*(45e-3/2)**2
eff_sphere = (incident_flux*mask_area)/Flux_out_of_fiber
print(f'efficiency of sphere is {eff_sphere[ind]}')

## Declare a mask hole size

In [ ]:
## The magnification factor is the ratio of the focal lengths
f_lsst = 10.3  # [m]
f_cbp = 0.635 # [m]
d_cbp = 24.1e-2 # [m] - so 24.1 cm
f_num_cbp = f_cbp/d_cbp

mag = f_lsst / f_cbp
print(f'Magnification is {f:03f}')
# so if we want a top-hat on the detector which X pixels in diameter
blob_diam_pix = 50  # diameter of blob on LSST focal plane in pixels
pixel_size = 10e-6  # [meters]
pinhole_diam =  blob_diam_pix * pixel_size / mag # [m]

print(f'A {blob_diam_pix} pixel ({blob_diam_pix*pixel_size*1e6:0.0f} [um]) diameter tophat on the LSST detector requires a {1e6*pinhole_diam:0.1f} [um] diameter hole on the CBP mask')

## Amount of Light exiting the CBP and hitting the detector for a perfect optical system

In [ ]:
E_lsst = np.pi * incident_flux * (np.pi*(pinhole_diam/2)**2) / (4*f_num_cbp**2)
print(f'Total energy in the tophat for a perfect system is {E_lsst[ind]:0.3e} [Watts]')

### CBP transmission is approximately 50% (vendor says 56, but coatings have probably eroded)

In [ ]:
CBP_system_efficiency =  0.56  # Excludes mask
LSST_system_efficiency = 0.65 * 0.45 # telescope then camera -- based on LSE-30, but needs better definition
flux_hitting_focal_plane = E_lsst * CBP_system_efficiency * LSST_system_efficiency # watts

print(f'Total energy in the tophat for a the actual system is {flux_hitting_focal_plane[ind]:0.3e} [Watts]')

### Convert from energy to photons - get photons per pixel

In [ ]:
flux_hitting_focal_plane_photons = flux_hitting_focal_plane / (h*c/(laser_wave*1e-9)) # photons per second
#want the number of photons per second hitting eachpixel.
#Flux_hitting_focal_plane_photons[ind]

In [ ]:
print(f'Flux per top-hat is {flux_hitting_focal_plane_photons[ind]:0.3e} [ph/s]')
#print(f'Flux per pixel is {flux_hitting_focal_plane_photons[ind]/:0.3e} [ph/s]')

# END OF CBP WORK FOR NOW - 2022-01-12

In [ ]:
total_number_of_pixels = (np.pi*(blob_diam_pix/2)**2)
# full_well = 130000 # electrons   - this is actually from DECam but we don't use it here

flux_per_pixel_per_second = flux_hitting_focal_plane_photons / total_number_of_pixels
print("Photons per pixel per second is %d at %d nm" % (flux_per_pixel_per_second[ind],laser_wave[ind]) )

In [ ]:
if 0 == 1:
    plt.plot(laser_wave,filterless_flux_per_pixel_per_second,'-g')
    plt.title( 'Filterless Single Laser Incident flux on individual detector pixels',fontsize=20)
    plt.ylabel('Filterless Flux [ph/pix/s]',fontsize=20)
    plt.xlabel('wavelength [nm]',fontsize=20)
    plt.xlim([300,1300])
    plt.text(600, 3200, '**Assumes NO FILTER**')
    plt.legend()
    #plt.ylim([0,1])
    plt.show()
    plt.close()

### Calculate time to complete a monochromatic flat

In [ ]:
# assume a required MEDIAN SNR for a given wavelength
med_SNR=100.0

# Find the wavelength to start/end a scan

# load a filter (but use total transmission since that is what actually
# matters and the filters have leaks that confuse the algorithm)
filter = 'u'  # ugrizy
filename='LSST_response_files/total_'+filter+'.dat'
total_trans000 = np.genfromtxt(filename, delimiter=' ',comments='#') # wavelength [nm], Transmission
#find nans and remove them
good = ~np.isnan(total_trans000[:,1])
total_trans0=np.transpose(np.array([total_trans000[good,0], total_trans000[good,1]]))


In [ ]:
#define the X% transmission points where we want to start/end the scan
min_trans=0.01 # so 1% transmission point

# Also find the wavelength of central bandpass transmission
# this is roughly the half way point of the filter, defined
# by the where the total transmission is greater than a given 
# percentage 

cen_ind, = np.where(total_trans0[:,1] > 0.1) # data with >10% throughput
central_trans_value=sorted(total_trans0[cen_ind,1])[int(round(cen_ind.size / 2))]
central_value_ind,=np.where(total_trans0[:,1] == central_trans_value)


In [ ]:
# smooth by a few nm to help find the desired transmission boundary % level
# helps ensure no double values occur
smoothed_filter_profile = np.convolve(total_trans0[:,1], [0,1,1,1,0], mode='same')

if 0==1:
    plt.plot(total_trans0[:,0],total_trans0[:,1],'-r')
    plt.show()
    plt.close()
    
#separate into red and blue sides of the filter for ease of code comprehension
blue_half_of_filter = smoothed_filter_profile[0:central_value_ind[0]]
red_half_of_filter = smoothed_filter_profile[central_value_ind[0]::]
#find closest value to min_trans on blue and red edge
blue_side_edge_wave_ind = (np.abs(blue_half_of_filter-min_trans)).argmin()
red_side_edge_wave_ind = (np.abs(red_half_of_filter-min_trans)).argmin()

print('minimum scan wavelength is is %d nm, based on minimum total transmission point of %0.2f%%' 
      % (total_trans0[blue_side_edge_wave_ind,0],min_trans) )
print('maximum scan wavelength is is %d nm, based on minimum total transmission point of %0.2f%%' 
      % (total_trans0[central_value_ind+red_side_edge_wave_ind,0],min_trans) )

In [ ]:
# define filter bandpass range to compute flat based on value of min_trans
filter_bandpass = np.arange(total_trans0[blue_side_edge_wave_ind,0],
                            total_trans0[central_value_ind[0]+red_side_edge_wave_ind,0])

# load detector efficiency
det_filename='LSST_response_files/detector_120227.dat'
detector0 = np.genfromtxt(det_filename, delimiter='',comments='#') # wavelength [nm], Transmission
#interpolate to filter_bandpass
detector_efficiency = scipy.interpolate.griddata(detector0[:,0],detector0[:,1], filter_bandpass, method='linear')

# load the filter curve
filename='LSST_response_files/filter_'+filter+'.dat'
filter_trans0 = np.genfromtxt(filename, delimiter=' ',comments='#') # wavelengt
#get filter transmission for this bandpass
filter_transmission = scipy.interpolate.griddata(filter_trans0[:,0],
                                                 filter_trans0[:,1], filter_bandpass, method='linear')

# determine filterless and detectorless count rate average
filterless_count_rate = scipy.interpolate.griddata(laser_wave,filterless_flux_per_pixel_per_second,
                                                   filter_bandpass, method='linear') 
count_rate = filterless_count_rate * filter_transmission * detector_efficiency
    
median_count_rate = np.median(count_rate)
print('median count rate for %s-band is %d photons/sec with the %s laser' % (filter ,median_count_rate,laser) )

# determine how long it will take to collect med_SNR^2 of photons
time = (med_SNR**2) / median_count_rate * filter_bandpass.size / 3600  # in hours
print('%s-band will take %0.2f hours for an median SNR of %d' % (filter,time,med_SNR) )

snr_per_wave=np.sqrt(count_rate* ((med_SNR**2) / median_count_rate))
integrated_snr= np.sqrt(sum(count_rate* ((med_SNR**2) / median_count_rate)))

print('%s-band will have an integrated SNR of %d ' % (filter, integrated_snr) )

# create SNR plot
if 1 == 1:
    plt.plot(filter_bandpass,snr_per_wave,'-g')
    plt.title( 'SNR of %s-band monochromatic flat' % (filter),fontsize=20)
    plt.ylabel('SNR',fontsize=20)
    plt.xlabel('wavelength [nm]',fontsize=20)
    #plt.legend()
    plt.show()
    plt.close()

# END HERE